In [1]:
import os
import requests
from dotenv import load_dotenv

# -----------------------------------------------------------
# Load from .env (local)
# -----------------------------------------------------------
load_dotenv()

# -----------------------------------------------------------
# If not found, try reading from secrets.toml (TOML style)
# -----------------------------------------------------------
def load_from_toml():
    try:
        import toml
        if os.path.exists(".streamlit/secrets.toml"):
            data = toml.load(".streamlit/secrets.toml")
            return data.get("GITHUB_TOKEN")
    except Exception:
        pass
    return None


GITHUB_TOKEN = os.getenv("GITHUB_TOKEN") or load_from_toml()

OWNER = "Trilokesh-Praxis-2023"
REPO = "Finalcial_App"

headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
}

base = f"https://api.github.com/repos/{OWNER}/{REPO}"

print("Token loaded:", GITHUB_TOKEN is not None)
print("-" * 60)

# 1. Repo access
r1 = requests.get(base, headers=headers)
print("Repo access:\n", r1.text)
print("-" * 60)

# 2. Read CSV
r2 = requests.get(f"{base}/contents/finance_data.csv", headers=headers)
print("Read CSV:\n", r2.text)
print("-" * 60)

# 3. Git ref
r3 = requests.get(f"{base}/git/ref/heads/main", headers=headers)
print("Git ref:\n", r3.text)
print("-" * 60)

# 4. Create blob
r4 = requests.post(
    f"{base}/git/blobs",
    headers=headers,
    json={"content": "hello", "encoding": "utf-8"},
)
print("Create blob:\n", r4.text)
print("-" * 60)


Token loaded: True
------------------------------------------------------------
Repo access:
 {"id":1104792547,"node_id":"R_kgDOQdnL4w","name":"Finalcial_App","full_name":"Trilokesh-Praxis-2023/Finalcial_App","private":false,"owner":{"login":"Trilokesh-Praxis-2023","id":146763912,"node_id":"U_kgDOCL9wiA","avatar_url":"https://avatars.githubusercontent.com/u/146763912?v=4","gravatar_id":"","url":"https://api.github.com/users/Trilokesh-Praxis-2023","html_url":"https://github.com/Trilokesh-Praxis-2023","followers_url":"https://api.github.com/users/Trilokesh-Praxis-2023/followers","following_url":"https://api.github.com/users/Trilokesh-Praxis-2023/following{/other_user}","gists_url":"https://api.github.com/users/Trilokesh-Praxis-2023/gists{/gist_id}","starred_url":"https://api.github.com/users/Trilokesh-Praxis-2023/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/Trilokesh-Praxis-2023/subscriptions","organizations_url":"https://api.github.com/users/Trilokesh-Praxis

In [10]:
import pandas as pd

file_path = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\finance_data_full.xlsx"

# -----------------------------
# Load
# -----------------------------
df = pd.read_excel(file_path)

# -----------------------------
# Clean columns
# -----------------------------
df['period'] = pd.to_datetime(df['period'], errors='coerce')
df['amount'] = pd.to_numeric(df['amount'], errors='coerce').fillna(0)

# -----------------------------
# Remove rows using '_row'
# -----------------------------
df = df[~df['_row'].astype(str).str.lower().isin(['yes', '1', 'true'])]

# -----------------------------
# Create MONTH
# -----------------------------
df['month'] = df['period'].dt.strftime('%Y-%m')

# -----------------------------
# Date-wise total
# -----------------------------
daily = (
    df.groupby('period', as_index=False)['amount']
      .sum()
      .sort_values('period')
)

# -----------------------------
# Running total on date
# -----------------------------
daily['running_total'] = daily['amount'].cumsum()

# -----------------------------
# Merge back
# -----------------------------
df = df.merge(daily[['period', 'running_total']], on='period', how='left')

# -----------------------------
# Fix duplicate running_total columns if existed
# -----------------------------
if 'running_total_x' in df.columns:
    df['running_total'] = df['running_total_y']
    df = df.drop(columns=['running_total_x', 'running_total_y'])

# -----------------------------
# Final cleanup
# -----------------------------
df['period'] = df['period'].dt.date
df = df.drop(columns=['_row'], errors='ignore')

# -----------------------------
# Save as CSV
# -----------------------------
output_path = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\finance_data_cleaned.csv"
df.to_csv(output_path, index=False)

print("✅ CSV created at:", output_path)


✅ CSV created at: C:\Users\trilo\OneDrive\Desktop\Report_Transactions\finance_data_cleaned.csv


In [2]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Build DB URL
DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

def test_connection():
    try:
        engine = create_engine(DATABASE_URL)
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1")).scalar()
            if result == 1:
                print("✔ Connection Successful! DB is reachable.")
    except Exception as e:
        print(f"❌ Connection Failed:\n{e}")

if __name__ == "__main__":
    test_connection()


✔ Connection Successful! DB is reachable.


# Data Check

In [3]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

engine = create_engine(DATABASE_URL)

df = pd.read_sql("SELECT * FROM finance_data order by period DESC", engine)

# Show everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(10)


,period,accounts,category,amount,month,_row,running_total
0,2026-01-11 00:00:00,UPI,Transport,340.0,None,NaN,NaN
1,2026-01-11 00:00:00,UPI,Party_Weekend,44.0,None,NaN,NaN
2,2026-01-09 00:00:00,UPI,Food,19.0,None,NaN,NaN
3,2026-01-09 00:00:00,UPI,Food,35.0,None,NaN,NaN
4,2026-01-09 00:00:00,UPI,Apparel,370.0,None,NaN,NaN
5,2026-01-09 00:00:00,UPI,Beauty,332.0,None,NaN,NaN
6,2026-01-09 00:00:00,UPI,Food,15.0,None,NaN,NaN
7,2026-01-09 00:00:00,UPI,Gift,65.0,None,NaN,NaN
8,2026-01-08 00:00:00,UPI,Food,23.0,None,NaN,NaN
9,2026-01-07 00:00:00,UPI,Food,16.0,None,NaN,NaN


In [4]:
engine = create_engine(DATABASE_URL)

df = pd.read_sql("SELECT period,category, AVG(amount) as avg_amount FROM finance_data GROUP BY category,period Having category = 'Rent' ORDER BY avg_amount DESC", engine)

# Show everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(25)


,period,category,avg_amount
0,2024-10-02 00:00:00,Rent,46000.0
1,2025-04-01 00:00:00,Rent,11686.0
2,2025-07-02 00:00:00,Rent,11679.0
3,2025-05-02 00:00:00,Rent,11670.0
4,2025-06-01 00:00:00,Rent,11649.0
5,2025-03-03 00:00:00,Rent,11596.0
6,2025-01-01 00:00:00,Rent,11517.0
7,2026-01-02 00:00:00,Rent,11505.0
8,2025-02-01 00:00:00,Rent,11498.0
9,2025-12-01 00:00:00,Rent,11400.0


In [5]:
engine = create_engine(DATABASE_URL)

df = pd.read_sql("SELECT period,category, AVG(amount) as avg_amount FROM finance_data GROUP BY period, category ORDER BY avg_amount DESC", engine)

# Show everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(25)


,period,category,avg_amount
0,2024-10-02 00:00:00,Rent,46000.0
1,2025-04-01 00:00:00,Rent,11686.0
2,2025-07-02 00:00:00,Rent,11679.0
3,2025-05-02 00:00:00,Rent,11670.0
4,2025-06-01 00:00:00,Rent,11649.0
5,2025-03-03 00:00:00,Rent,11596.0
6,2025-01-01 00:00:00,Rent,11517.0
7,2026-01-02 00:00:00,Rent,11505.0
8,2025-02-01 00:00:00,Rent,11498.0
9,2024-11-02 00:00:00,Rent,11400.0


In [6]:
print(3*(11686- 9800))

print(3*(11686- 6500))

print(((11686- 6500)/(11686- 9800)))


5658
15558
2.7497348886532342


# Recovery

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# ============================================================
# 🔐 Load Environment Variables
# ============================================================
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

# ============================================================
# 📂 Excel File Path
# ============================================================
file_path = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\3380.xlsx"

# ============================================================
# 🗄 Table Name (change if needed)
# ============================================================
TABLE_NAME = "finance_data"

# ============================================================
# 🚀 Push Excel → PostgreSQL
# ============================================================
def push_excel_to_db():
    # Read Excel
    df = pd.read_excel(file_path)

    # Optional: clean column names (recommended)
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[^\w_]", "", regex=True)
    )

    # Create DB engine
    engine = create_engine(DATABASE_URL)

    # Push to DB
    df.to_sql(
        TABLE_NAME,
        engine,
        if_exists="append",   # use "replace" if you want to overwrite
        index=False,
        method="multi",
        chunksize=5000
    )

    print(f"✅ {len(df)} rows pushed successfully to '{TABLE_NAME}'")

# ============================================================
# ▶ Run
# ============================================================
if __name__ == "__main__":
    push_excel_to_db()


In [ ]:
import subprocess

PG_BIN = r"C:\Program Files\PostgreSQL\16\bin"
CREATEDB = fr"{PG_BIN}\createdb.exe"
PG_RESTORE = fr"{PG_BIN}\pg_restore.exe"

BACKUP_FILE = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\Backup\3380.dat"
DB_NAME = "report_mqba_restore"

subprocess.run([CREATEDB, DB_NAME], capture_output=True)
subprocess.run([PG_RESTORE, "-d", DB_NAME, BACKUP_FILE], check=True)

print("✅ Database restored")
